In [280]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm

Lx, Ly = (4, 4)
L = Lx*Ly
t = 0.11
# construct Hamiltonian
latt = qib.lattice.TriangularLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 2)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=200)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

hloc1 = construct_ising_local_term(J, 0, 0, ndim=2)
hloc2 = g*(np.kron(X, I2)+np.kron(I2, X))/6
perms_1 = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12]]
perms_2 = [[0, 5, 10, 15, 3, 4, 9, 14, 2, 7, 8, 13, 1, 6, 11, 12], [5, 10, 15, 0, 4, 9, 14, 3, 7, 8, 13, 2, 6, 11, 12, 1]]
perms_3 = [[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15], [4, 8, 12, 0, 5, 9, 13, 1, 6, 10, 14, 2, 7, 11, 15, 3]]

"""Vlist_start = [YZ, V1, V2, V1, YZ]*2 +  [YZ, V3, V4, V3, YZ]  + [YZ, V1, V2, V1, YZ]*2
Vlist_reduced = [V1, V2, V1]*2 +  [V3, V4, V3]  + [V1, V2, V1]*2

perms_extended = [[perms_1[0]]] + [perms_1]*3 + [[perms_1[0]], [perms_2[0]]] +\
                    [perms_2]*3 + [[perms_2[0]], [perms_3[0]]] + [perms_3]*3 + [[perms_3[0]], [perms_2[0]]] +\
                    [perms_2]*3 + [[perms_2[0]], [perms_1[0]]] + [perms_1]*3 + [[perms_1[0]]]
perms_ext_reduced = [perms_1]*3  + [perms_2]*3 + [perms_3]*3 + [perms_2]*3 + [perms_1]*3
control_layers = [0, 4, 5, 9, 10, 14, 15, 19, 20, 24] # 10 control layers

state = np.array(random_statevector(2**L).data)
print("Trotter error: ", np.linalg.norm(ansatz_sparse(Vlist_start, L, perms_extended, state) - expm_multiply(
    -1j * t * hamil, state), ord=2))"""

'Vlist_start = [YZ, V1, V2, V1, YZ]*2 +  [YZ, V3, V4, V3, YZ]  + [YZ, V1, V2, V1, YZ]*2\nVlist_reduced = [V1, V2, V1]*2 +  [V3, V4, V3]  + [V1, V2, V1]*2\n\nperms_extended = [[perms_1[0]]] + [perms_1]*3 + [[perms_1[0]], [perms_2[0]]] +                    [perms_2]*3 + [[perms_2[0]], [perms_3[0]]] + [perms_3]*3 + [[perms_3[0]], [perms_2[0]]] +                    [perms_2]*3 + [[perms_2[0]], [perms_1[0]]] + [perms_1]*3 + [[perms_1[0]]]\nperms_ext_reduced = [perms_1]*3  + [perms_2]*3 + [perms_3]*3 + [perms_2]*3 + [perms_1]*3\ncontrol_layers = [0, 4, 5, 9, 10, 14, 15, 19, 20, 24] # 10 control layers\n\nstate = np.array(random_statevector(2**L).data)\nprint("Trotter error: ", np.linalg.norm(ansatz_sparse(Vlist_start, L, perms_extended, state) - expm_multiply(\n    -1j * t * hamil, state), ord=2))'

In [282]:
"""with h5py.File(f"./results/triangularTFIM_ccU_SPARSE_103_Lx4Ly4_t0.25_layers25_niter5_rS1_2hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]"""

with h5py.File(f'./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers9_niter10_rS1_2hloc.hdf5') as f:
    Vlist =  f["Vlist"][:]
perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
                    [perms_2] + [[perms_2[0]], [perms_3[0]]] + [perms_3] + [[perms_3[0]]] 
perms_ext_reduced = [perms_1]  + [perms_2] + [perms_3]
control_layers = [0, 2, 3, 5, 6, 8]

In [283]:
"""
    Test operations.
"""

Vlist_reduced = []
for i, V in enumerate(Vlist):
    if i not in control_layers:
        Vlist_reduced.append(V)

state = random_statevector(2**L).data
print("Trotter error of the optimized point: ", (np.linalg.norm(ansatz_sparse(Vlist, L, perms_extended, state) - expm_multiply(
    1j * t * hamil, state), ord=2) + np.linalg.norm(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state) - expm_multiply(
    -1j * t * hamil, state), ord=2))/2)

print('infidelity: ', 1-(state_fidelity(ansatz_sparse(Vlist, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)) + state_fidelity(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state), expm_multiply(
    -1j * t * hamil, state)))/2)

Trotter error of the optimized point:  0.03470905491985431
infidelity:  0.0013255611570761605


In [284]:
sys.path.append("../../src/controlled_unitary_optimizer")
sys.path.append("../../src/brickwall_ansatz")
from optimize_3q import optimize_3q 
from utils_3q import make_controlled, random_unitary

Xlists_opt = {}
#perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
#perms_qc = [[0, 1], [0, 2]]
perms_qc = [[0, 1], [0, 2]]

for i in control_layers:
    cU = make_controlled(Vlist[i])
    f_best, err_best, Glist_best = (0, 2, None)
    for _ in range(10):
        Xlist_start = [random_unitary(4) for i in range(len(perms_qc))]
        Xlist, f_iter, err_iter = optimize_3q(L, cU, Xlist_start, perms_qc, niter=1000)
        if err_iter[-1] < err_best:
            f_best, err_best, Xlist_best = (f_iter[-1], err_iter[-1], Xlist)
    print("Best f: ", f_best)
    print("Best err: ", err_best)
    Xlists_opt[i] = Xlist_best

Best f:  -7.997488103527692
Best err:  0.03692717348990398
Best f:  -7.997146260974857
Best err:  0.03869509911442172
Best f:  -7.996622833949025
Best err:  0.04275652398606278
Best f:  -7.996263214414816
Best err:  0.0436739178467932
Best f:  -7.996797416024911
Best err:  0.04158885694159643
Best f:  -7.99643472355916
Best err:  0.04287120103959836


In [285]:
# Save the optimal gates.

for i in control_layers:
    with h5py.File(f"./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers25_niter5_rS1_DECOMPOSE_n{len(perms_qc)}_layer{i}.hdf5", "w") as file:
        file.create_dataset(f"Xlist_{i}", data=Xlists_opt[i])


In [286]:
#perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
perms_qc = [[0, 1], [0, 2]]
Xlists_opt = {}
for i in control_layers:
    with h5py.File(f"./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers25_niter5_rS1_DECOMPOSE_n{len(perms_qc)}_layer{i}.hdf5", "r") as file:
        Xlists_opt[i] = file[f"Xlist_{i}"][:]

In [287]:
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp

qc = qiskit.QuantumCircuit(L+2)
qc.x(L)
qc.x(L+1)
for i, V in enumerate(Vlist):
    layer = i
    if i in control_layers:
        Glist = Xlists_opt[i]
        qc_3 = qiskit.QuantumCircuit(3)
        for j, G in enumerate(Glist):
            qc_3.unitary( G, (3-1-perms_qc[j][1], 3-1-perms_qc[j][0]))

        for perm in perms_extended[layer]:
            for j in range(L//2):
                qc.append(qc_3.to_gate(), [L-perm[2*j]-1, L-perm[2*j+1]-1, L + (j%2)])
        
    else:
        for perm in perms_extended[layer]:
            for j in range(L//2):
                qc.unitary(V, [L-perm[2*j]-1, L-perm[2*j+1]-1])
qc.x(L)
qc.x(L+1)


ccU_cxs_05= []
ccU_errs_05 = []
for t in [t]:
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+2)
    qc_ext1.initialize(state, [i for i in range(L)])
    for i in range(int(t/t)):
        qc_ext1.append(qc.to_gate(), [i for i in range(L+2)])
    backend = Aer.get_backend("statevector_simulator")
    sv1 = execute(transpile(qc_ext1), backend).result().get_statevector().data

    qc_ext2 = qiskit.QuantumCircuit(L+2)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    qc_ext2.x(L+1)
    for i in range(int(t/t)):
        qc_ext2.append(qc.to_gate(), [i for i in range(L+2)])
    backend = Aer.get_backend("statevector_simulator")
    sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data


    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    ket_0_e = np.eye(1)
    ket_1_e = np.eye(1)
    for i in range(2):
        ket_0_e = np.kron(ket_0_e, ket_0)
        ket_1_e = np.kron(ket_1_e, ket_1)
    exact_v1 = np.kron(ket_0_e, expm_multiply(1j * t * hamil, state))
    exact_v2 = np.kron(ket_1_e, expm_multiply(-1j * t * hamil, state))
    err = (np.linalg.norm(sv1-exact_v1, ord=2) + np.linalg.norm(sv2-exact_v2, ord=2))/2
    #err = 1-(state_fidelity(sv1, exact_v1)+state_fidelity(sv2, exact_v2))/2
    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
    count_ops = dag.count_ops()
    print(count_ops)
    
    #ccU_errs_05.append(err)
    #ccU_cxs_05.append(count_ops['unitary'])
    print(f"t={t}, Gate Count: ", count_ops['unitary'], " State infidelity: ", err)
    #print("Fidelity: ", 1-err)

{'initialize': 1, 'sx': 8, 'unitary': 144}
t=0.11, Gate Count:  144  State infidelity:  0.05351100306678979


In [206]:
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


qc = qiskit.QuantumCircuit(L+1)
qc.x(L)
for i, V in enumerate(Vlist):
    layer = i
    if i in control_layers:
        Glist = Xlists_opt[i]
        qc_3 = qiskit.QuantumCircuit(3)
        for j, G in enumerate(Glist):
            qc_3.unitary( G, (3-1-perms_qc[j][1], 3-1-perms_qc[j][0]))

        for perm in perms_extended[layer]:
            for j in range(L//2):
                qc.append(qc_3.to_gate(), [L-perm[2*j]-1, L-perm[2*j+1]-1, L])
        
    else:
        
        for perm in perms_extended[layer]:
            for j in range(L//2):
                qc.unitary(V, [L-perm[2*j]-1, L-perm[2*j+1]-1])
qc.x(L)


ccU_cxs_05= []
ccU_errs_05 = []
for t in [t]:
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    for i in range(int(t/t)):
        qc_ext1.append(qc.to_gate(), [i for i in range(L+1)])

    backend = Aer.get_backend("statevector_simulator")
    sv1 = execute(transpile(qc_ext1), backend).result().get_statevector().data

    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    for i in range(int(t/t)):
        qc_ext2.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t * hamil, state))
    err = (np.linalg.norm(sv1-exact_v1, ord=2) + np.linalg.norm(sv2-exact_v2, ord=2))/2
    #err = (1-state_fidelity(sv1, exact_v1) + 1-state_fidelity(sv2, exact_v2))/2
    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
    count_ops = dag.count_ops_longest_path()
    print(count_ops)
    
    ccU_errs_05.append(err)
    ccU_cxs_05.append(count_ops['unitary'])
    print(f"t={t}, Gate Count: ", count_ops['unitary'], " State infidelity: ", err)
    print("Infidelity: ", err) 

{'sx': 4, 'unitary': 96}
t=0.12, Gate Count:  96  State infidelity:  0.09243446955199613
Infidelity:  0.09243446955199613


In [122]:
"""
    Now here is to compare the performance of the ccU circuit
    with the 1st and 2nd order Trotter circuits, in terms of 
    gate count vs Trotter error. I demonstrate it on L=10 system.
"""

from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


def controlled_trotter(c1, L, Lx, Ly, J, g, perms_1, perms_2, perms3, dag=False, nsteps=1):
    if c1/(2*nsteps) > .125:
        nsteps = int(np.ceil(c1/(2 * .125)))
    t = c1/(2*nsteps)

    indices = [0, 1]
    coeffs = [1, 1]
    #indices = oc.SplittingMethod.suzuki(2, 1).indices
    #coeffs = oc.SplittingMethod.suzuki(2, 1).coeffs
    perms_ext = [perms_1, perms_2, perms_3]*len(indices)
    
    hloc1 = g*(np.kron(X, I2)+np.kron(I2, X))/6
    hloc2 = construct_ising_local_term(J, 0, 0, ndim=2)
    hlocs = (hloc1, hloc2)

    K = []
    for i, perms in enumerate(perms_ext):
        perm = perms[0]
        K_layer = [None for _ in range(L)]
        for j in range(len(perm)//2):
            K_layer[perm[2*j]] = CYGate
            K_layer[perm[2*j+1]] = CZGate
        K.append(K_layer)

    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hlocs[i]))
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1], label='str')
        Vlist_gates.append(qc2)


    qc = qiskit.QuantumCircuit(L+1)
    for n in range(nsteps):
        for layer, qc_gate in enumerate(Vlist_gates):

            perms = perms_1
            qc.x(L)
            for j in range(L):
                if K[3*layer][j]:
                    qc.append(K[3*layer][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
            qc.x(L)
            for j in range(L):
                if K[3*layer][j]:
                    qc.append(K[3*layer][j](), [L, L-1-j])
            qc.x(L)

            perms = perms_2
            qc.x(L)
            for j in range(L):
                if K[3*layer+1][j]:
                    qc.append(K[3*layer+1][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])

            qc.x(L)
            for j in range(L):
                if K[3*layer+1][j]:
                    qc.append(K[3*layer+1][j](), [L, L-1-j])
            qc.x(L)

            perms = perms_3
            qc.x(L)
            for j in range(L):
                if K[3*layer+2][j]:
                    qc.append(K[3*layer+2][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
            qc.x(L)
            for j in range(L):
                if K[3*layer+2][j]:
                    qc.append(K[3*layer+2][j](), [L, L-1-j])
            qc.x(L)
    return qc

    
for t in [t]:
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    qc_ext1.append(controlled_trotter(2*t, L, Lx, Ly, J, g, perms_1, perms_2, perms_3).to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv1_T = execute(transpile(qc_ext1), backend).result().get_statevector().data
    
    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    qc_ext2.append(controlled_trotter(2*t, L, Lx, Ly, J, g, perms_1, perms_2, perms_3).to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2_T = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t * hamil, state))
    #err = (np.linalg.norm(sv1_T-exact_v1, ord=2) + np.linalg.norm(sv2_T-exact_v2, ord=2))/2
    err = 1-(state_fidelity(sv1_T, exact_v1) + state_fidelity(sv2_T, exact_v2))/2

    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
    count_ops = dag.count_ops()
    print(count_ops['cx']+count_ops['unitary'])

    #trotter2_cxs_01.append(count_ops['cx']+count_ops['unitary'])
    #trotter2_errs_01.append(err)

    print(f"t={t}, State Infid: ", count_ops['cx']+count_ops['unitary'])
    print("Infidelity: ", err)

288
t=0.125, State Infid:  288
Infidelity:  0.1623222649094459


In [ ]:
# Compare perms_qc length 3 to Trotter 4th order r=1, Trotter 2nd order r=1 & r=2 and Trotter 1st order r=2 & r=3.